In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import os
%matplotlib inline
#plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler,FunctionTransformer,StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold,StratifiedShuffleSplit, train_test_split, learning_curve,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, roc_curve
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
rnd_state = 0

In [4]:
# create a database connection
sqluser = 'maxim'
dbname = 'maxim'
schema_name = 'mimiciii'

In [5]:
# Connect to local postgres version of mimic
connect = psycopg2.connect(dbname=dbname, user=sqluser)
cursor = connect.cursor()
cursor.execute('SET search_path to {}'.format(schema_name))

In [6]:
# Load in the query from file
query='SELECT * FROM OASIS'
oasis = pd.read_sql_query(query, connect)
oasis = oasis.dropna(axis=0)
oasis = oasis.loc[(oasis.urineoutput < 20000) & (oasis.urineoutput > 0) & 
                  (oasis.temp > 30) & (oasis.temp < 45) & (oasis.icustay_age_group == 'adult')]
oasis['preiculos_hours'] = oasis.preiculos.dt.total_seconds()/3600

In [7]:
oasis.head()

,subject_id,hadm_id,icustay_id,icustay_age_group,hospital_expire_flag,icustay_expire_flag,oasis,oasis_prob,age,age_score,preiculos,preiculos_score,gcs,gcs_score,heartrate,heartrate_score,meanbp,meanbp_score,resprate,resprate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score,preiculos_hours
0,55973,152234,200001,adult,0,0,42,0.305849,61.0,6,7 days 03:02:12,1,14.0,3.0,134.0,6.0,60.000000,2.0,32.0,6.0,36.388889,2.0,250.0,10.0,0,0,0,6,171.036667
1,27513,163557,200003,adult,0,0,35,0.152892,48.0,3,0 days 02:48:04,3,15.0,0.0,122.0,3.0,179.000000,3.0,39.0,6.0,36.388889,2.0,3652.0,0.0,1,9,0,6,2.801111
2,10950,189514,200006,adult,0,0,32,0.109623,54.0,6,0 days 00:01:14,5,15.0,0.0,73.0,0.0,61.000000,2.0,27.0,1.0,36.166666,2.0,1955.0,1.0,1,9,0,6,0.020556
3,20707,129310,200007,adult,0,0,26,0.054187,43.0,3,0 days 00:01:37,5,15.0,0.0,104.0,1.0,50.666698,3.0,29.0,1.0,36.388889,2.0,1295.0,5.0,0,0,0,6,0.026944
4,29904,129607,200009,adult,0,0,25,0.048012,47.0,3,-1 days +23:49:32,5,15.0,0.0,106.0,1.0,60.000000,2.0,17.5,0.0,34.599998,4.0,1570.0,1.0,1,9,1,0,-0.174444


In [26]:
#X = oasis_adults.drop(['subject_id', 'hadm_id', 'icustay_id', 'icustay_age_group', 'hospital_expire_flag',
#                       'icustay_expire_flag', 'oasis', 'oasis_prob', 'age_prob'], axis=1)
num_features = ['age', 'preiculos_hours', 'gcs', 'heartrate', 'meanbp', 'resprate', 'temp', 'urineoutput']
cat_features = ['mechvent', 'electivesurgery']
X = oasis[num_features + cat_features]
#y = oasis.icustay_expire_flag
y = oasis.hospital_expire_flag
X_train_orig, X_test_orig, y_train, y_test = train_test_split(X, y, test_size=0.2, \
                                                    shuffle=True, stratify=y, random_state=rnd_state)


In [27]:
print('Shape of training set: {}'.format(X_train_orig.shape))
print('Shape of test set: {}'.format(X_test_orig.shape))
print('Number of positives in training set: {}'.format(y_train.sum()))
print('Number of positives in test set: {}'.format(y_test.sum()))

Shape of training set: (38797, 10)
Shape of test set: (9700, 10)
Number of positives in training set: 4480
Number of positives in test set: 1120


In [28]:
num_impute = ('num_impute', SimpleImputer(missing_values=np.nan, strategy='median'))
scale = ('scale', MinMaxScaler())
num_transformer = Pipeline([num_impute, scale])

cat_impute = ('cat_impute', SimpleImputer(strategy='constant', fill_value=2))
#onehot = ('onehot', OneHotEncoder(handle_unknown='error'))
cat_transformer = Pipeline(steps=[cat_impute])

preprocessor = ColumnTransformer(transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)])

X_train = preprocessor.fit_transform(X_train_orig)
X_test = preprocessor.transform(X_test_orig)

In [29]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

In [30]:
from torch.utils import data
class Dataset(data.Dataset):
    
  def __init__(self, X, y):
        'Initialization'
        self.X = X
        self.y = y

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.X)

  def __getitem__(self, index):
        'Generates one sample of data'

        X = self.X[index]
        y = self.y[index]

        return X, y

In [31]:
# obtain training indices that will be used for validation
num_train = len(X_train)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(Dataset(X_train,y_train.values), batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(Dataset(X_train,y_train.values), batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(Dataset(X_test,y_test.values), batch_size=batch_size, 
    num_workers=num_workers)

In [32]:
# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (10)
        hidden_1 = 128
        # linear layer (10 -> hidden_1)
        self.fc1 = nn.Linear(10, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, 1)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 10)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        #x = x.view(-1, 1)
        return x

# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=10, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [39]:
# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (10)
        hidden = 256
        # linear layer (10 -> hidden_1)
        self.fc1 = nn.Linear(10, hidden)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        # linear layer (n_hidden -> 1)
        self.fc4 = nn.Linear(hidden, 1)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 10)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        # add dropout layer
        x = self.dropout(x)
        # add output layer
        x = self.fc4(x)
        x = torch.sigmoid(x)
        #x = x.view(-1, 1)
        return x

# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=10, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [40]:
# specify loss function (categorical cross-entropy)
#criterion = nn.CrossEntropyLoss()

criterion = nn.BCELoss()
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

In [41]:
# number of epochs to train the model
n_epochs = 100

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data.float())
        # calculate the loss
        target = target.view(-1,1)
        weight=compute_sample_weight(class_weight='balanced', y=target)
        weight=torch.Tensor(weight)
        weight=weight.view(-1,1)
        criterion.weight = weight
        loss = criterion(output.float(), target.float())
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation
    for data, target in valid_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data.float())
        # calculate the loss
        target = target.view(-1,1)
        weight=compute_sample_weight(class_weight='balanced', y=target)
        weight=torch.Tensor(weight)
        weight=weight.view(-1,1)
        criterion.weight = weight
        loss = criterion(output.float(), target.float())
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.498792 	Validation Loss: 0.120103
Validation loss decreased (inf --> 0.120103).  Saving model ...
Epoch: 2 	Training Loss: 0.461771 	Validation Loss: 0.111797
Validation loss decreased (0.120103 --> 0.111797).  Saving model ...
Epoch: 3 	Training Loss: 0.450467 	Validation Loss: 0.111022
Validation loss decreased (0.111797 --> 0.111022).  Saving model ...
Epoch: 4 	Training Loss: 0.442393 	Validation Loss: 0.110829
Validation loss decreased (0.111022 --> 0.110829).  Saving model ...
Epoch: 5 	Training Loss: 0.437578 	Validation Loss: 0.108793
Validation loss decreased (0.110829 --> 0.108793).  Saving model ...
Epoch: 6 	Training Loss: 0.434813 	Validation Loss: 0.107968
Validation loss decreased (0.108793 --> 0.107968).  Saving model ...
Epoch: 7 	Training Loss: 0.435212 	Validation Loss: 0.108048
Epoch: 8 	Training Loss: 0.436272 	Validation Loss: 0.110035
Epoch: 9 	Training Loss: 0.432894 	Validation Loss: 0.106762
Validation loss decreased (0.107968 --> 0.

In [42]:
model = Net()
model.load_state_dict(torch.load('model.pt'))
model.eval()

Net(
  (fc1): Linear(in_features=10, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [43]:
#model.eval() # prep model for evaluation
test_loss = 0.0
predictions = []
for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data.float())
    # calculate the loss
    #target = target.view(-1,1)
    #loss = criterion(output.float(), target.float())
    # update test loss 
    #test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    #_, pred = torch.max(output, 1)
    # compare predictions to true label
    #predictions += pred
    predictions += output.view(1,-1).tolist()[0]
    #predictions += output.tolist()
    #correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    #for i in range(batch_size):
    #    label = target.data[i]
    #    class_correct[label] += correct[i].item()
    #    class_total[label] += 1
binary_predictions = np.round(predictions)

In [44]:
F1 = f1_score(y_test, binary_predictions)
auroc = roc_auc_score(y_test, predictions)
precision = precision_score(y_test, binary_predictions)
recall = recall_score(y_test, binary_predictions)
res = 'TEST SET f1: {0:.2f} auroc: {1:.2f} precision: {2:.2f} recall: {3:.2f}'.format(F1,auroc,precision,recall)
print(res)

TEST SET f1: 0.40 auroc: 0.82 precision: 0.28 recall: 0.69
